## Biblioteki

In [1]:
import json
import os
import requests
from bs4 import BeautifulSoup

## Funkcja do ekstrakcji zawartości  ze strony html

In [ ]:

def extract(ancestor, selector=None, attribute=None, returns_list=False):
    if selector:
        if returns_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.get_text().strip() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().strip() 
        except AttributeError:
            return None
    if attribute :
        return ancestor[attribute].strip()
    return ancestor.get_text().strip

: 

## Słownik reprezentujący strukturę 

In [7]:
selectors = {
                "opinion_id": ( None,'data-entry-id'),
                "author": ("span.user-post__author-name",),
                "recommendation": ("span.user-post__author-recomendation > em",),
                "rating": (" span.user-post__score-count"),
                "content": ( "div.user-post__text",),
                "pros" : ( "div.review-feature__title--positives ~ div.review-feature__item", None, True),
                "cons" : ( "div.review-feature__title--negatives ~ div.review-feature__item", None, True),
                "useful": ( 'span[id^="vote-yes"]',),
                "useless": ('span[id^="vote-no"]',),
                "post_date": ('span.user-post__published > time:nth-child(1)', "datetime"),
                "purchase_date": ('span.user-post__published > time:nth-child(2)', "datetime")
            }

## Link do pierwszej strony z opiniami o wskazanym produkcie

In [2]:
product_id = "114700014"

#url = f"https://www.ceneo.pl/114700014#tab=reviews" #https://www.ceneo.pl/114700014
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"# /opinie-1


# Ekstrakcja składowych pojedynczych opinii
| Składowa | Selektor | Zmienna |
|----------|----------|---------|
| id opinii| ["data-entry-id"] | opinion_id|
| autor    | span.user-post__author-name |author |
| rekomendacja| span.user-post__author-recomendation > em |recommendation|
| gwiazdki | span.user-post__score-count| rating|
| treść    | div.user-post__text|content | 
| lista zalet |div.review-feature__title review-feature__title--positives | pros|
| lista wad | div.review-feature__title review-feature__title--negatives| cons|
| dla ilu przydatna | span[id^="vote-yes"]| useful|
| dla ilu nieprzydatna| span[id^="vote-no"]| useless|
| data wystawienia | span.user-post__published > time:nth-child(1)["datetime"]| post_date|
|data zakupu | span.user-post__published > time:nth-child(2)["datetime"] | purchase_rate |

In [ ]:
single_opinion = {
"opinion_id" : opinion["data-entry-id"],
"author": opinion.select_one("span.user-post__author-name").get_text().strip(),
"recommendation": opinion.select_one("span.user-post__author-recomendation > em").get_text().strip(),
"rating": opinion.select_one("span.user-post__score-count").get_text().strip(),
"content": opinion.select_one("div.user-post__text").get_text().strip(),
"pros": [p.get_text().strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
"cons": [c.get_text().strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
"useful" : opinion.select_one("span[id^='votes-yes']").get_text().strip(),
"useless": opinion.select_one("span[id^='votes-no']").get_text().strip(),
"post_date": opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip(),
"purchase_date": opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"].strip(),
}



: 

In [14]:
all_opinions = []
while(url):
    print(url)
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
            single_opinion = {
                key: extract(opinion,*value)
                    for key, value in selectors.items()
            }
            all_opinions.append(single_opinion)
       
    try:
        url = "https://www.ceneo.pl"+extract(page_dom, "a.pagination__next", "href")
    except TypeError:
        url = None

https://www.ceneo.pl/114700014#tab=reviews


NameError: name 'extract' is not defined

In [ ]:
#print(single_opinion)
if not os.path.exists("opinions"):
    os.mkdir("opinions")

#print(json.dumps(all_opinions, indent=4, ensure_ascii=False))
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)

: 

In [9]:
len(opinions)

2